In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import os
import datetime as dt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import cmocean
import json
import f90nml
from collections import OrderedDict
from scipy.stats.mstats import gmean

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display



%matplotlib inline


In [2]:
PSdf=pd.read_excel('PugetSoundMesoZoopBiomassDF_Publication.xlsx',engine='openpyxl',parse_dates=['Sample Date','dtUTC'])

In [3]:
SoGdf=pd.read_excel('SoGMesoZoopBiomassDF_Publication.xlsx',engine='openpyxl',parse_dates= ['Date','dtUTC'])

In [4]:
SoGdf.keys()

Index(['Unnamed: 0', 'Key', 'region_name', 'Station', 'Lon', 'Lat', 'Date',
       'dtUTC', 'Twilight', 'Net_Type', 'Mesh_Size(um)', 'Z_lower', 'Z_upper',
       'Bottom Depth(m)', 'Amphipoda', 'Thecostraca', 'Decapoda',
       'Euphausiacea', 'Calanoida', 'Aphragmophora', 'Leptothecate',
       'Trachylina', 'Anthoathecatae', 'Phragmophora', 'Year', 'Month', 'YD',
       'Total', 'Amphipods', 'Euphausiids', 'Calanoids', 'Hydrozoans'],
      dtype='object')

In [5]:
SoGdf.rename(columns={'Aphragmophora':'Chaetognaths'},inplace=True)

In [6]:
SoGdf.keys()

Index(['Unnamed: 0', 'Key', 'region_name', 'Station', 'Lon', 'Lat', 'Date',
       'dtUTC', 'Twilight', 'Net_Type', 'Mesh_Size(um)', 'Z_lower', 'Z_upper',
       'Bottom Depth(m)', 'Amphipoda', 'Thecostraca', 'Decapoda',
       'Euphausiacea', 'Calanoida', 'Chaetognaths', 'Leptothecate',
       'Trachylina', 'Anthoathecatae', 'Phragmophora', 'Year', 'Month', 'YD',
       'Total', 'Amphipods', 'Euphausiids', 'Calanoids', 'Hydrozoans'],
      dtype='object')

In [7]:
SoGdf.drop(labels=['Station','Twilight', 'Net_Type', 'Mesh_Size(um)','Bottom Depth(m)', 'Amphipoda', 'Euphausiacea', 'Calanoida',
       'Leptothecate','Chaetognaths', 'Trachylina','Thecostraca','Decapoda', 'Anthoathecatae',
       'Phragmophora', 'Year', 'Amphipods',
       'Euphausiids', 'Calanoids', 'Hydrozoans','Month', ],axis=1,inplace=True) #removed 'YD' from drop list

In [8]:
SoGdf.keys()

Index(['Unnamed: 0', 'Key', 'region_name', 'Lon', 'Lat', 'Date', 'dtUTC',
       'Z_lower', 'Z_upper', 'YD', 'Total'],
      dtype='object')

In [9]:
PSdf.keys()

Index(['Unnamed: 0', 'Key', 'Station', 'Site Name', 'Basin', 'Sub Basin',
       'Lat', 'Lon', 'Sample Date', 'Sample Year', 'Sample Month',
       'Sample Time', 'Mesh Size', 'Diameter (cm)', 'Z', 'Z_lower', 'Z_upper',
       'dtUTC', 'CALANUS MARSHALLAE', 'CALANUS PACIFICUS', 'EUCALANUS',
       'EUCALANUS BUNGII', 'METRIDIA PACIFICA', 'NEOCALANUS PLUMCHRUS',
       'CYPHOCARIS CHALLENGERI', 'CALLIOPIUS PACIFICUS', 'GAMMARIDEA',
       'THEMISTO PACIFICA', 'EUPHAUSIA PACIFICA', 'THYSANOESSA RASCHII',
       'CHAETOGNATHA', 'CLYTIA GREGARIA', 'AEQUOREA VICTORIA',
       'AGLANTHA DIGITALE', 'BARNACLES', 'FABIA SUBQUADRATA',
       'CANCER PRODUCTUS', 'CANCRIDAE', 'Cancridae Lg', 'Cancridae Sm',
       'GLEBOCARCINUS OREGONENSIS', 'LOPHOPANOPEUS BELLUS',
       'METACARCINUS GRACILIS', 'METACARCINUS MAGISTER', 'PINNIXA',
       'PINNOTHERES', 'PUGETTIA', 'Year', 'YD', 'Total'],
      dtype='object')

***rename columns***

In [10]:
PSdf.drop(labels=['Site Name','Station','Sub Basin', 'Sample Year', 'Sample Month',
       'Sample Time','Mesh Size', 'Diameter (cm)','Z',
        'CALANUS MARSHALLAE', 'CALANUS PACIFICUS', 'EUCALANUS',
       'EUCALANUS BUNGII', 'METRIDIA PACIFICA', 'NEOCALANUS PLUMCHRUS',
       'CYPHOCARIS CHALLENGERI', 'CALLIOPIUS PACIFICUS', 'GAMMARIDEA',
       'THEMISTO PACIFICA', 'EUPHAUSIA PACIFICA', 'THYSANOESSA RASCHII',
       'CHAETOGNATHA', 'CLYTIA GREGARIA', 'AEQUOREA VICTORIA',
       'AGLANTHA DIGITALE', 'BARNACLES', 'FABIA SUBQUADRATA',
       'CANCER PRODUCTUS', 'CANCRIDAE', 'Cancridae Lg', 'Cancridae Sm',
       'GLEBOCARCINUS OREGONENSIS', 'LOPHOPANOPEUS BELLUS',
       'METACARCINUS GRACILIS', 'METACARCINUS MAGISTER', 'PINNIXA',
       'PINNOTHERES', 'PUGETTIA', 'Year', ],axis=1,inplace=True) #removed 'YD' from drop list

In [11]:
PSdf.keys()

Index(['Unnamed: 0', 'Key', 'Basin', 'Lat', 'Lon', 'Sample Date', 'Z_lower',
       'Z_upper', 'dtUTC', 'YD', 'Total'],
      dtype='object')

In [12]:
PSdf.rename(columns={'Basin':'region_name','Sample Date':'Date'},inplace=True)

In [13]:
SoGdf.keys()

Index(['Unnamed: 0', 'Key', 'region_name', 'Lon', 'Lat', 'Date', 'dtUTC',
       'Z_lower', 'Z_upper', 'YD', 'Total'],
      dtype='object')

In [14]:
PSdf.keys()

Index(['Unnamed: 0', 'Key', 'region_name', 'Lat', 'Lon', 'Date', 'Z_lower',
       'Z_upper', 'dtUTC', 'YD', 'Total'],
      dtype='object')

In [15]:
SalishSeadf=pd.concat([PSdf, SoGdf], ignore_index=True)

In [16]:
SalishSeadf

,Unnamed: 0,Key,region_name,Lat,Lon,Date,Z_lower,Z_upper,dtUTC,YD,Total
0,0,010218ELIV1151,Bellingham Bay,48.63795,-122.5694,2018-01-02,110.0,0,2018-01-02 19:51:00,2,7.707972
1,1,010818SKETV1058,South Sound,47.15243,-122.6586,2018-01-08,122.0,0,2018-01-08 18:58:00,8,10.461341
2,2,011119ELIV1052,Bellingham Bay,48.63795,-122.5694,2019-01-11,114.0,0,2019-01-11 18:52:00,11,5.705943
3,3,011315CAMV1330,Whidbey Basin,48.05901,-122.3873,2015-01-13,170.0,0,2015-01-13 21:30:00,13,14.000355
4,4,011315MUKV1415,Whidbey Basin,47.97166,-122.3222,2015-01-13,190.0,0,2015-01-13 22:15:00,13,27.594135
...,...,...,...,...,...,...,...,...,...,...,...
2500,1341,UBC9999099001401,Juan de Fuca,48.30000,-124.0000,2018-08-27,104.0,0,2018-08-28 01:45:00,240,20.694478
2501,1342,UBC9999099001601,Central Strait of Georgia,49.22800,-123.4700,2016-08-30,309.0,0,2016-08-30 19:00:00,243,31.761374
2502,1343,UBC9999099001801,Central Strait of Georgia,49.25000,-123.6670,2017-12-04,375.0,0,2017-12-04 20:00:00,338,2.046578
2503,1344,UBC9999099001901,Central Strait of Georgia,49.25000,-123.6670,2018-01-15,375.0,0,2018-01-15 20:00:00,15,3.514918


In [17]:
#create new columns
SalishSeadf['Month']=[ii.month for ii in SalishSeadf['dtUTC']]
SalishSeadf['Year']=[ii.year for ii in SalishSeadf['dtUTC']]

In [18]:
SalishSeadf['Total mg m2']=(SalishSeadf['Total'])*SalishSeadf['Z_lower']

In [19]:
SalishSeadf

,Unnamed: 0,Key,region_name,Lat,Lon,Date,Z_lower,Z_upper,dtUTC,YD,Total,Month,Year,Total mg m2
0,0,010218ELIV1151,Bellingham Bay,48.63795,-122.5694,2018-01-02,110.0,0,2018-01-02 19:51:00,2,7.707972,1,2018,847.876968
1,1,010818SKETV1058,South Sound,47.15243,-122.6586,2018-01-08,122.0,0,2018-01-08 18:58:00,8,10.461341,1,2018,1276.283548
2,2,011119ELIV1052,Bellingham Bay,48.63795,-122.5694,2019-01-11,114.0,0,2019-01-11 18:52:00,11,5.705943,1,2019,650.477556
3,3,011315CAMV1330,Whidbey Basin,48.05901,-122.3873,2015-01-13,170.0,0,2015-01-13 21:30:00,13,14.000355,1,2015,2380.060423
4,4,011315MUKV1415,Whidbey Basin,47.97166,-122.3222,2015-01-13,190.0,0,2015-01-13 22:15:00,13,27.594135,1,2015,5242.885742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,1341,UBC9999099001401,Juan de Fuca,48.30000,-124.0000,2018-08-27,104.0,0,2018-08-28 01:45:00,240,20.694478,8,2018,2152.225764
2501,1342,UBC9999099001601,Central Strait of Georgia,49.22800,-123.4700,2016-08-30,309.0,0,2016-08-30 19:00:00,243,31.761374,8,2016,9814.264412
2502,1343,UBC9999099001801,Central Strait of Georgia,49.25000,-123.6670,2017-12-04,375.0,0,2017-12-04 20:00:00,338,2.046578,12,2017,767.466563
2503,1344,UBC9999099001901,Central Strait of Georgia,49.25000,-123.6670,2018-01-15,375.0,0,2018-01-15 20:00:00,15,3.514918,1,2018,1318.094437


In [20]:
import netCDF4 as nc

In [21]:
fdict={'biol_T':1,'grid_T':1}
start_date = dt.datetime(2021,1,1) #previous results used (2012,1,1) 
end_date = dt.datetime(2021,8,26) #previous results used (2016,12,31)
flen=1 # number of days per model output file. always 1 for 201905 and 201812 model runs
namfmt='nowcast' # for 201905 and 201812 model runs, this should always be 'nowcast'
# filemap is dictionary of the form variableName: fileType, where variableName is the name
# of the variable you want to extract and fileType designates the type of 
# model output file it can be found in (usually ptrc_T for biology, grid_T for temperature and 
# salinity)
filemap={'microzooplankton':'biol_T','mesozooplankton':'biol_T'}
# fdict is a dictionary mappy file type to its time resolution. Here, 1 means hourly output
# (1h file) and 24 means daily output (1d file). In certain runs, multiple time resolutions 
# are available
fdict={'biol_T':1,'grid_T':1}

In [22]:
PATH= '/home/jvalenti/scratch/run_SHEM/months-SHEM/' 

In [23]:
SalishSeadf

,Unnamed: 0,Key,region_name,Lat,Lon,Date,Z_lower,Z_upper,dtUTC,YD,Total,Month,Year,Total mg m2
0,0,010218ELIV1151,Bellingham Bay,48.63795,-122.5694,2018-01-02,110.0,0,2018-01-02 19:51:00,2,7.707972,1,2018,847.876968
1,1,010818SKETV1058,South Sound,47.15243,-122.6586,2018-01-08,122.0,0,2018-01-08 18:58:00,8,10.461341,1,2018,1276.283548
2,2,011119ELIV1052,Bellingham Bay,48.63795,-122.5694,2019-01-11,114.0,0,2019-01-11 18:52:00,11,5.705943,1,2019,650.477556
3,3,011315CAMV1330,Whidbey Basin,48.05901,-122.3873,2015-01-13,170.0,0,2015-01-13 21:30:00,13,14.000355,1,2015,2380.060423
4,4,011315MUKV1415,Whidbey Basin,47.97166,-122.3222,2015-01-13,190.0,0,2015-01-13 22:15:00,13,27.594135,1,2015,5242.885742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,1341,UBC9999099001401,Juan de Fuca,48.30000,-124.0000,2018-08-27,104.0,0,2018-08-28 01:45:00,240,20.694478,8,2018,2152.225764
2501,1342,UBC9999099001601,Central Strait of Georgia,49.22800,-123.4700,2016-08-30,309.0,0,2016-08-30 19:00:00,243,31.761374,8,2016,9814.264412
2502,1343,UBC9999099001801,Central Strait of Georgia,49.25000,-123.6670,2017-12-04,375.0,0,2017-12-04 20:00:00,338,2.046578,12,2017,767.466563
2503,1344,UBC9999099001901,Central Strait of Georgia,49.25000,-123.6670,2018-01-15,375.0,0,2018-01-15 20:00:00,15,3.514918,1,2018,1318.094437


In [24]:
M_path = '/home/jvalenti/MOAD/'
mask_path = M_path+'grid2/mesh_mask202108_TDV.nc'

In [25]:
data=et.matchData(SalishSeadf,filemap,fdict,mask_path,mod_start=start_date,mod_end=end_date,mod_nam_fmt='shem',mod_basedir=PATH,method='vertNet');

mod_start=2021-01-01 00:00:00
mod_end=2021-08-26 00:00:00


In [26]:
data

,Unnamed: 0,Key,region_name,Lat,Lon,Date,Z_lower,Z_upper,dtUTC,YD,Total,Month,Year,Total mg m2,j,i,mod_microzooplankton,mod_mesozooplankton,k_upper,k_lower


In [27]:
data['mod_total']=data['mod_microzooplankton']+data['mod_mesozooplankton']


In [28]:
data['mod_mesozooplanktonDI']=(data['mod_mesozooplankton'])*data['Z_lower']

In [29]:
# define log transform function with slight shift to accommodate zero values
def logt(x):
  return np.log10(x+.001)

In [30]:
# define inverse log transform with same shift
def logt_inv(y):
    return 10**y-.001

In [31]:
#Create columns for Log10 transformations
data['L10Total']=logt(data['Total'])
data['L10TotalDI']=logt(data['Total mg m2']/1000)


#Convert model values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
data['L10mod_microzooplankton']=logt(data['mod_microzooplankton']*5.7*12)
data['L10mod_mesozooplankton']=logt(data['mod_mesozooplankton']*5.7*12)
data['L10mod_total']=logt(data['mod_total']*5.7*12)
data['L10mod_mesozooplanktonDI']=logt(data['mod_mesozooplanktonDI']*5.7*12/1000)


In [32]:
data['logt_invL10Total']=logt_inv(data['L10Total'])
data['logt_invL10TotalDI']=logt_inv(data['L10TotalDI'])

data['logt_invL10mod_mesozooplanktonDI']=logt_inv(data['L10mod_mesozooplanktonDI'])

In [33]:
#data.to_csv('SalishSea_MesozoopModelvsObsMatches_2015to2019_Eval202111.csv') #pandas to csv and split the notebook with the remainder of the notebook; name effectively because you will reuse this